In [ ]:
def read_csv(csv_file_path, debug):
        """
            Given a path to a csv file, return a matrix (list of lists)
            in row major.
        """
        res = []
        with open(csv_file_path, debug) as csv_file:
            lines = csv_file.readlines()
            for line in lines:
                row = []
                values = line.strip('\n\r"').split(',')
                #NEW VERSION
                for val in values:
                    if val != '':
                        row.append(convert_str(val))
                if row != []:
                    res.append(row)
        return res

In [ ]:
#NEW VERSION

#Get csv file straightened out with Jena's code
my_data = read_csv('WorkforceUtilizationSummaryReportApril2019.csv','r')


#Get Project codes, names, and contractor names

new_data = []             #Version with project code, project name, and contractor names truncated
full_names_data = []      #Version with just complete contractor names grouped per project


#List of project codes 
projects = []

#List of truncated project names
projects_names = []

#Lists of things to ignore while searching for contractor names
ignore_list1 = [[],'Total for Contractor',['Project Name:'],'Journey',['Journey'], 'Journey', ['Total for Contractor'] ]
ignore_list2 = ['Journey','Total for Contractor','Project Name:','Total  Journey Hours','Apprentice','Total Journey Hours']


#Entry number in processed csv
i = 0

#Huge while loop to go through data
while i < len(my_data):
        
    if my_data[i][0] == 'Project Name:':
        
        if my_data[i+3][0] not in projects:
            
            projects_names.append(my_data[i+1][0])
            projects.append(my_data[i+3][0])
          
            #To group contractors per project
            new_entry = [my_data[i+1][0], my_data[i+3][0]]
            project_full_names = []
            
            i += 4
            
            #Another huge while loop
            while i < len(my_data):
                
                #Just many instances of text that goes before any contractor name
                
                if my_data[i][0] == 'Subtotal' and my_data[i+1][0] not in ignore_list2:
                    #Check if the next entry isn't a Construction Trade name or a number
                    if not my_data[i+1][0].isupper() and not my_data[i+1][0][0].isnumeric():
                        contractor_name = my_data[i+1][0]
                        #Get truncated contractor name
                        new_entry.append(contractor_name)
                        
                        #Check if name has been truncated
                        if len(my_data[i+1]) > 1 and my_data[i+1][1] not in ignore_list1 and\
                        my_data[i+1][1] not in ignore_list2:
                            contractor_name = contractor_name + my_data[i+1][1]
                        #Add complete name to list with complete contractor names
                        project_full_names.append(contractor_name)
                    
                elif my_data[i][0] == 'Male' and my_data[i+1] not in ignore_list1 and my_data[i+1][0] not in ignore_list2:
                    #Check if the next entry isn't a Construction Trade name or a number
                    if not my_data[i+1][0].isupper() and not my_data[i+1][0][0].isnumeric():
                        contractor_name = my_data[i+1][0]
                        #Get truncated contractor name
                        new_entry.append(contractor_name)
                        #Check if name has been truncated
                        if len(my_data[i+1]) > 1 and my_data[i+1][1] not in ignore_list1 and\
                        my_data[i+1][1] not in ignore_list2:
                            contractor_name = contractor_name + my_data[i+1][1]
                        #Add complete name to list with complete contractor names
                        project_full_names.append(contractor_name)
                    else:
                        #This is a special case because this contractor has all caps in its name, unlike other contractor
                        #names and like Construction Trade names, so...
                        if my_data[i+1][0] == 'AAA ROYAL CONSTRUCTION LLC':
                            #Get truncated contractor name
                            new_entry.append(my_data[i+1][0])
                            #Add complete name to list with complete contractor names (which is the same)
                            project_full_names.append(my_data[i+1][0])
                
                elif (len(my_data[i]) > 1) and my_data[i][1] == 'Subtotal' and my_data[i+1][0] not in ignore_list2:
                    #Check if the next entry isn't a Construction Trade name or a number
                    if not my_data[i+1][0].isupper()and not my_data[i+1][0][0].isnumeric():
                        contractor_name = my_data[i+1][0]
                        #Get truncated contractor name
                        new_entry.append(contractor_name)
                        #Check if name has been truncated
                        if len(my_data[i+1]) > 1 and my_data[i+1][1] not in ignore_list1 and\
                        my_data[i+1][1] not in ignore_list2:
                            contractor_name = contractor_name + my_data[i+1][1]
                        #Add complete name to list with complete contractor names
                        project_full_names.append(contractor_name)
                
                #We've possibly reached another project
                elif my_data[i][0] == 'Project Name:':
                    #Check if this isn't just a repeat due to structure of PDFs
                    if my_data[i+3][0] not in projects:
                        i -= 2
                        break
                        
                    
                i += 1
            
            #Add the grouped by project data to a big list
            new_data.append(new_entry)
            #Add the grouped by project data to a big list
            full_names_data.append(project_full_names)
    
    i += 1    
               
         
        
        
new_data

In [ ]:
#NEW VERSION

#Some important info to record
num_projects = len(projects)
current_project = 0
i = 0

#The almost final output!
list_for_df = []


#Horribly huge while loop
while i < len(my_data):
    
    #Check if the current entry equals the project code we're looking for
    if my_data[i][0] == projects[current_project]:
        
        #Some info for later
        current_project_code = my_data[i][0]
        current_project_name = my_data[i-2][0][len(current_project_code):]   #Delete project code from project name
        #Special case for some project codes that differ from the one listed in the project name
        if current_project_name[0] == ' ':
            current_project_name = current_project_name[1:]
            
        i += 1
        
        #Some info to record
        current_contractor = 2                                  #Because in list, first two entries are project         
        num_contractors = len(new_data[current_project])-2      #code and project name and then come contractor names
            
        while i < len(my_data):
            
            #Check if we're not on the last contractor for the current project
            if current_contractor-2 != num_contractors-1:
                
                #Check if the current entry matches the contractor we're looking for
                if my_data[i][0] == new_data[current_project][current_contractor]:
                        
                    i += 1
                    
                    #Go through data until reach the next contractor name
                    while my_data[i][0] != new_data[current_project][current_contractor + 1]:
                        
                        #Watch for number entries which are useless in this search
                        if type(my_data[i][0]) != str:
                        
                            i += 1
                        #Check that entry isn't a project code
                        elif my_data[i][0] != projects[current_project]:
                            #Check that data is all caps or that it has a substring that's in Construction Trade names
                            if my_data[i][0].isupper() or 'Class' in my_data[i][0]:
                                #Check if the Trade name hasn't been cut into two separate lists
                                if my_data[i+1][0].isupper() or 'Class' in my_data[i+1][0]:
                                    
                                    current_construction_trade = my_data[i][0] + my_data[i+1][0]
                                    
                                    i += 1
                                #Check if the Trade name hasn't been sperated into two separate entries of the list        
                                elif len(my_data[i])>1 and (my_data[i][1].isupper() or 'Class' in my_data[i][1]):
                                    current_construction_trade = my_data[i][0] + my_data[i][1]
                                
                                else:                              
                                    current_construction_trade = my_data[i][0]
                                
                                #Create an entry with all the relevant info: 
                                #      Project Code, Project Name, Contractor Name, Construction Trade
                                list_for_df.append([current_project_code,current_project_name,\
                                                    full_names_data[current_project][current_contractor-2],\
                                                    current_construction_trade])
                                i += 1
                            else:
                                i += 1
                        else:
                            i += 1
                    
                    #Update current contractor
                    current_contractor += 1
                
                else:
                    i+=1
            
            #Check that we're not on the last project
            elif current_project != num_projects-1:
                
                #Check if the current entry matches the contractor we're looking for    
                if my_data[i][0] == new_data[current_project][current_contractor]:
                        
                    i += 1
                    
                    #Go through data until reach the next project code
                    while my_data[i][0] != projects[current_project+1]:
                        
                        #Watch for number entries which are useless in this search    
                        if type(my_data[i][0]) != str:
                            
                            i += 1
                            
                        #Check that entry isn't a project code
                        elif my_data[i][0] != projects[current_project]:
                            #Check that data is all caps or that it has a substring that's in Construction Trade names
                            if my_data[i][0].isupper() or 'Class' in my_data[i][0]:
                                
                                #Check if the Trade name hasn't been cut into two separate lists
                                if my_data[i+1][0].isupper() or 'Class' in my_data[i+1][0]:
                                    
                                    current_construction_trade = my_data[i][0] + my_data[i+1][0]
                                    
                                    i += 1
                                    
                                #Check if the Trade name hasn't been sperated into two separate entries of the list          
                                elif len(my_data[i])>1 and (my_data[i][1].isupper() or 'Class' in my_data[i][1]):
                                    current_construction_trade = my_data[i][0] + my_data[i][1]
                                
                                else:                              
                                    current_construction_trade = my_data[i][0]
                                
                                #Create an entry with all the relevant info: 
                                #      Project Code, Project Name, Contractor Name, Construction Trade
                                list_for_df.append([current_project_code,current_project_name,\
                                                    full_names_data[current_project][current_contractor-2],\
                                                    current_construction_trade])
                                i += 1
                            else:
                                i += 1
                                
                        else:
                            i += 1
                    break
                    
                else:
                    i += 1
            
            #So we're on the last contractor of the last project!
            else:
                
                #Check if the current entry matches the contractor we're looking for
                if my_data[i][0] == new_data[current_project][current_contractor]:
                        
                    i += 1
                    
                    #Go through data until reach the end of the data
                    while i < len(my_data):
                        
                        #Watch for number entries which are useless in this search
                        if type(my_data[i][0]) != str:
                            
                            i += 1
                            
                        #Check that entry isn't a project code
                        elif my_data[i][0] != projects[current_project]:
                            
                            #Check that data is all caps or that it has a substring that's in Construction Trade names
                            if my_data[i][0].isupper() or 'Class' in my_data[i][0]:
                                
                                #Check if the Trade name hasn't been cut into two separate lists
                                if my_data[i+1][0].isupper() or 'Class' in my_data[i+1][0]:
                                    
                                    current_construction_trade = my_data[i][0] + my_data[i+1][0]
                                    
                                    i += 1
                                
                                #Check if the Trade name hasn't been sperated into two separate entries of the list        
                                elif len(my_data[i])>1 and (my_data[i][1].isupper() or 'Class' in my_data[i][1]):
                                    current_construction_trade = my_data[i][0] + my_data[i][1]
                                
                                else:                              
                                    current_construction_trade = my_data[i][0]
                                
                                #Create an entry with all the relevant info: 
                                #      Project Code, Project Name, Contractor Name, Construction Trade
                                list_for_df.append([projects[current_project],new_data[current_project][0],\
                                                    full_names_data[current_project][current_contractor-2],\
                                                    current_construction_trade])
                                i += 1
                            else:
                                i+=1
                        else:
                            i += 1
                    break
                else:
                    i+=1
                    
        #Update Current Project
        current_project +=1

    else:
        i += 1  
                    
                
                    

            
#Ta-da!!                
list_for_df, len(list_for_df)

In [ ]:
import pandas as pd

#Create a nice pandas dataframe
df = pd.DataFrame(list_for_df, columns = [ 'Project_Code', 'Project_Name', 'Contractor_Name', 'Construction_Trade' ]) 

#Look at it!
df